<a href="https://colab.research.google.com/github/wiesnervos/codelabs/blob/master/Avera_Aggregator_UAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Using the Avera AI FRTB-SA Aggregator from Python.

The Avera AI FRTB-SA Aggregator is a Google cloud service which calculates the regulatory capital under the [Basel framework](https://www.bis.org/basel_framework/index.htm?tldate=20220316).  This Colab demonstrates how to access the FRTB-SA Aggregator service from Python.

This Colab performs some User Acceptance Testing (UAT) to verify behaviour and results. 

### To run the code in each cell highlight the cell and press <Ctrl+Return>. ###


# Avera User Acceptance Testing (UAT)

Below we implement UAT Tests for the AVERA AI FRTB Aggeration Service. This includes the following tests:

**Snapshots**:


*   Create/Delete/Manipulate/Publish/List
*   Snapshot Formatting Errors
*   Verify Formatting of uploads
*   Uploading Of Sensitivities
* 

**Calculations**: 
*   Snapshot Not Exist Error Messages

     




In [ ]:
#@title Imports and Google Drive authetication (for test files)
# Testing Variables
TOTAL_EXPECTED_SNAPSHOTS = 131

# Portfolio Test file 1 (scratchpad-0.json1)
test_file_id_1 = '1xUBZ91xzJddEnkjFR_YXAl_vJW49u-B0'
# Portfolio Test file 2 (scratchpad-1.json1)
# Incorrectly formatted JSON
test_file_id_2 = '1nwS8MbkveaGryQzsQOEz_DL56WkkdP41'
# Portfolio Test file 3 (scratchpad-2.json1)
# Contains incorrect risk class: 'RANDOM_CLASS'
test_file_id_3 = '1KUmKcPHtrkcNY5CmD1wSNKPYG9hw7OGK'

# Portfolio Test file 3 (10000 trades)(scratchpad-4.jsonl)
# Used to test repeatability of results - section 4
test_file_id_4 = '1JgLTtKQydDyve0pbclu0cFCuGP0LRdUB'

# Portfolio Test file 4 (2 trades - unittest tests)
test_file_id_5 = '1FZ_r6fTG71qCbC9LFfUdnhxWGWsRE_bT'

# Two trade portfolio for testing sensitivity only calculations
# (two_trade_portfolio.json)
two_trade_portfolio = '1b_gKMeobamHlTlq179yreRn7FcG8LxMn'

# Three trade portfolio for testing DRC, RRAO and Sensitivity calculations
# (three_trade_portfolio.json)
three_trade_portfolio = '1u5Yx1k1trtFBinDfamMmkOWPRiTncB9Z'

# Set a tester ID to label snapshot names appropriately.
tester_name = 'KPMG'

# Import libraries and magics.
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
from IPython.display import Markdown, display

import enum
import json
import logging
import os
import time
import pandas as pd
import pandas_gbq
from typing import Any, Dict, Optional, Tuple, Union
import requests
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Better viszualization of portfolio data in Pandas.
!apt-get -qq install && pip install -q flat_table
import flat_table

# Authenticate and create the PyDrive client. 
# This only needs to be done once per notebook when it first 
# starts up.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Markdown printing.
def printmd(string):
    display(Markdown(string))

# Silence catching warning for Auth2. 
import logging
logging.getLogger('googleapiclient.discovery_cache').setLevel(logging.ERROR)
logging.getLogger('googleapiclient.discovery').setLevel(logging.CRITICAL)

In [ ]:
#@title Load Python Utilities Wrapping FRTB_SA REST API 
"""Simple client for FRTB service."""
class TokenType(enum.Enum):
  """Type of auth token."""
  AUTHORIZATION_CODE = 0
  REFRESH = 1


class FrtbError(RuntimeError):
  """Error returned by FRTB service."""
  pass


class FrtbClient:
  """Simple client for FRTB service."""

  def __init__(self,
               client_id: str,
               client_secret: str,
               token: str,
               token_type: TokenType = TokenType.AUTHORIZATION_CODE,
               base_url: str = "https://staging.aggregator.avera.area120.com/"):
    """Initializer.

    Args:
      client_id: Client id, which can be found in GCP console.
      client_secret: Client secret, which can be found in GCP console.
      token: Token for authorization.
        On first usage, in your browser proceed to
        https://accounts.google.com/o/oauth2/v2/auth?client_id=<client_id>&response_type=code&scope=openid%20email&access_type=offline&redirect_uri=urn:ietf:wg:oauth:2.0:oob
        Use the displayed token, with `token_type=TokenType.AUTHORIZATION_CODE`.
        After the first API call, the client will print out the refresh token,
        which can be subsequently with `token_type=TokenType.REFRESH` to speed
        up the flow.
      token_type: One of TokenType enum values, see above.
      base_url: Base url of the FRTB service. Staging API is used by default.
    """
    self._base_url = base_url
    self._auth = _Auth(client_id, client_secret, token, token_type)

  def list_snapshots(self,
                     page_size: int = 100,
                     page_token: str = "",
                     descending_order=False) -> Dict[Any, Any]:
    """Lists existing snapshots.

    Args:
      page_size: The maximum number of items to return.
      page_token: The next_page_token value returned from a previous request,
        if any.
      descending_order: Whether to sort in descending order by id.

    Returns:
      A parsed json object with response contents.
      TODO(b/154799237): replace these with response messages (after v2
      migration).
    """
    data = {
        "page_size": page_size,
        "page_token": page_token,
        "order_by": "id desc" if descending_order else "id asc"
    }
    return self._request("v2eap/frtb/snapshots", data=data)

  def create_scratchpad(self, name: str, title: str) -> Dict[Any, Any]:
    """Creates a scratchpad.

    Args:
      name: The REST resource name of the scratchpad.
      title: The display name to use for the eventual snapshot.

    Returns:
      A parsed json object with response contents.
    """
    return self._request("v2eap/frtb/scratchpads",
                         method="POST",
                         data={"name": name, "title": title})

  def add_to_scratchpad(self, scratchpad_id: int,
                        trades_json: str) -> Dict[Any, Any]:
    """Adds trades a scratchpad.

    Args:
      scratchpad_id: Scratchpad id (obtained via
        `create_scratchpad(...)["id"]`).
      trades_json: A json string with an array of TradeAndRiskValues objects.

    Returns:
      A parsed json object with response contents.
    """
    # Avoid parsing json and stringifying it back just to add the id.
    data = f'{{"id": {scratchpad_id}, "data": {trades_json}}}'  # pylint: disable=g-inconsistent-quotes
    return self._request(f"v2eap/frtb/scratchpads/id={scratchpad_id}:batchPut",
                         method="POST",
                         data=data)

  def publish_scratchpad(self, scratchpad_id: int) -> Dict[Any, Any]:
    """Publishes a scratchpad.

    Args:
      scratchpad_id: Scratchpad id (obtained via
        `create_scratchpad(...)["id"]`).

    Returns:
      A parsed json object with response contents.
    """
    return self._request(f"v2eap/frtb/scratchpads/id={scratchpad_id}:publish",
                         method="POST",
                         data={"id": scratchpad_id})

  def delete_scratchpad(self, scratchpad_id):
    """Deletes a scratchpad."""
    return self._request(f"v2eap/frtb/scratchpads/id={scratchpad_id}",
                         method="DELETE",
                         data={"id": scratchpad_id})

  def delete_snapshot(self, snapshot_id):
    """Deletes a snapshot."""
    return self._request(f"v2eap/frtb/snapshots/id={snapshot_id}",
                         method="DELETE",
                         data={"id": snapshot_id})
    
  def label_snapshot(self, snapshot_id, label):
    """Labels a snapshot."""
    return self._request(f"v2eap/frtb/snapshots/{snapshot_id}:addLabel",
                         method="POST",
                         data={"id": snapshot_id, "label": label})
  def relabel_snapshot(self, snapshot_id, label):
    """Update snapshot label."""
    return self._request(f"v2eap/frtb/snapshots/{snapshot_id}:updateLabel",
                         method="POST",
                         data={"id": snapshot_id, "label": label})

  def unlabel_snapshot(self, snapshot_id, label_key):
    """Delete snapshot label."""
    return self._request(f"v2eap/frtb/snapshots/{snapshot_id}:removeLabel",
                         method="POST",
                         data={"id": snapshot_id, "labelKey": label_key})
  
  def list_portfolios(self, snapshot_id, page_size: int = 100,
                      page_token: str = "") -> Dict[Any, Any]:
    """List trades in a snapshot and portfolio.

    Args:
      snapshot_id: a snapshot ID.
      page_size: The maximum number of items to return.
      page_token: The next_page_token value returned from a previous request,
        if any.
    """
    return self._request(f"/v2eap/frtb/snapshots/{snapshot_id}/portfolios",
                         method="GET",
                         data={"id": snapshot_id, 
                               "pageSize": page_size,
                               "pageToken": page_token})
    
  def list_trades(self, snapshot_id, portfolio_id, page_size: int = 100,
                  page_token: str = "") -> Dict[Any, Any]:
    """List trades in a snapshot and portfolio.

    Args:
      snapshot_id: a snapshot ID.
      portfolio_id: a portfolio ID.
      page_size: The maximum number of items to return.
      page_token: The next_page_token value returned from a previous request,
        if any.
    """
    return self._request(f"v2eap/frtb/snapshots/{snapshot_id}/portfolios/{portfolio_id}/trades",
                         method="GET",
                         data={"snapshotID": snapshot_id, 
                               "portfolioId": portfolio_id,
                               "pageSize": page_size,
                               "pageToken": page_token})

  def get_trades(self, snapshot_id, account_id, trade_id):
    """Get trades from an account."""
    return self._request(f"v2eap/frtb/snapshots/{snapshot_id}/accounts/{account_id}/trades/{trade_id}",
                         method="GET",
                         data={"id": snapshot_id, "account_id": account_id,
                               "trade_id": trade_id})

  def calculate_frtb_sa(self,
                        snapshot_id: int,
                        portfolio_id: int = 0) -> Dict[Any, Any]:
    """Calculates the FRTB-SA regulatory capital.

    Args:
     snapshot_id: The snapshot id to take the sensitivities and risk factors
       from.
     portfolio_id: The portfolio id to calculate the capital requirement for.

    Returns:
      A parsed json object with response contents.
    """
    return self._request(f"v2eap/frtb/snapshots/{snapshot_id}/portfolios/"
                         f"{portfolio_id}/aggregate:calculateFrtbSa")
    
  def calculate_frtb_sa_with_overrides(self,
                        snapshot_id: int,
                        portfolio_id: int = 0,
                        overrides: list = []) -> Dict[Any, Any]:
    """Calculates the FRTB-SA regulatory capital with overrides to specified
    trades.

    Args:
     snapshot_id: The snapshot id to take the sensitivities and risk factors
       from.
     portfolio_id: The portfolio id to calculate the capital requirement for.
     overrides: A list containing Trade and Risk values with where each element
                has the format:
                {
                  'trade' : {'tradeId' : int},
                  'sensitivityRiskFactors' : [{
                    'id' : FrtbRiskId,
                    'value' : value
                  }],
                  'rraoRiskFactors' : [{
                    'riskType' : enum,
                    'notional' : float
                  }],
                  'drcRiskfactors' : [{
                    'drcRiskFactorId' : DrcRiskFactorId,
                    'notional' : float,
                    'pnl' : float
                  }]
                }

    Returns:
      A parsed json object with response contents.
    """
    return self._request(f'v2eap/frtb/snapshots/{snapshot_id}/portfolios/{portfolio_id}/aggregate:overrideTrades',
                method = 'POST',
                data={'tradeOverrides' : {
                    'tradeAndRiskValues' : overrides
                }})

  def publish_snapshot_from_jsonl_file(
      self,
      path: str,
      name: str,
      title: str,
      timeout_between_requests_ms: int = 500) -> int:
    """Publishes a snapshot from a given json-lines file.

    Calls `create_scratchpad`, repeatedly calls `add_to_scratchpad` and finally,
    `publish_scratchpad`. Each line in the file must be a json array of
    TradeAndRiskValues objects. Each line will be treated as a batch, i.e. sent
    in a separate `add_to_scratchpad` request.

    Args:
      path: Path to a json lines file with TradeAndRiskValues objects.
      name: See `create_scratchpad`.
      title: See `create_scratchpad`.
      timeout_between_requests_ms: timeout between `add_to_scratchpad` requests.

    Returns:
       Id of the published snapshot.
    """
    print("Creating a scratchpad")
    scratchpad_id = self.create_scratchpad(name, title)["id"]
    with open(path, "r") as file:
      for i, trades_json in enumerate(file):
        if i != 0:
          time.sleep(timeout_between_requests_ms / 1000.0)
        print(f"Uploading batch {i + 1}")
        self.add_to_scratchpad(scratchpad_id, trades_json)
    print("Publishing")
    self.publish_scratchpad(scratchpad_id)
    return scratchpad_id

  def _request(
      self,
      endpoint: str,
      method: str = "GET",
      data: Optional[Union[str, Dict[Any, Any]]] = None) -> Dict[Any, Any]:
    """Wraps requests.request() with authorization etc."""
    url = self._base_url + endpoint
    if isinstance(data, dict):
      data = json.dumps(data)
    def try_request(id_token: str) -> Tuple[requests.Response, float]:
      t = time.time()
      response = requests.request(
          url=url,
          method=method,
          data=data,
          headers={"Authorization": f"Bearer {id_token}"})
      return response, time.time() - t

    response, elapsed = try_request(self._auth.fetch_id_token())
    if response.status_code == 401:
      # The token has probably expired.
      logging.info("Refreshing the token...")
      response, elapsed = try_request(self._auth.fetch_id_token(refresh=True))

    content = response.content.decode("utf-8")
    if response.ok:
      logging.info(f"Response from {url}:\n{content}\n"
                   f"Time elapsed: {round(elapsed, 2)} seconds.")
    else:
      raise FrtbError(f"Error calling {url}:\n{content}")

    return json.loads(content)


# TODO(b/156461543): Make the auth flow more user-friendly and use
#  google_auth_oauthlib.
class _Auth:
  """Manages the auth flow."""

  def __init__(self, client_id: str, client_secret: str, token: str,
               token_type: TokenType):
    self._client_id = client_id
    self._client_secret = client_secret
    self._token = token
    self._token_type = token_type
    self._id_token = None

  def fetch_id_token(self, refresh=False) -> str:
    """Fetches bearer id token.

    Args:
      refresh: If True, discards the saved token.

    Returns:
      Id token.
    """
    if not refresh and self._id_token is not None:
      return self._id_token
    if self._token_type == TokenType.AUTHORIZATION_CODE:
      self._id_token, self._token = (
          self._fetch_refresh_and_id_tokens_using_auth_code())
      self._token_type = TokenType.REFRESH
      print(f"Your refresh token is {self._token}\n"
            f"You can use it next time you create FrtbClient to speed up the "
            f"authentication flow.")
    else:
      self._id_token = self._fetch_id_token_using_refresh_token()
    logging.info(f"Acquired id token: {self._id_token}")
    return self._id_token

  def _fetch_refresh_and_id_tokens_using_auth_code(self) -> Tuple[str, str]:
    """Fetches refresh and id tokens using authorization token."""
    assert self._token_type == TokenType.AUTHORIZATION_CODE
    data = json.dumps({
        "client_id": self._client_id,
        "client_secret": self._client_secret,
        "redirect_uri": "urn:ietf:wg:oauth:2.0:oob",
        "grant_type": "authorization_code",
        "code": self._token
    })
    response = requests.post("https://www.googleapis.com/oauth2/v4/token",
                             data=data)
    if not response.ok:
      raise FrtbError(
          f"Authentication error: {response.content.decode('utf-8')}")

    content = json.loads(response.content)
    return content["id_token"], content["refresh_token"]

  def _fetch_id_token_using_refresh_token(self) -> str:
    """Fetches id token using refresh token."""
    assert self._token_type == TokenType.REFRESH
    data = json.dumps({
        "client_id": self._client_id,
        "client_secret": self._client_secret,
        "refresh_token": self._token,
        "grant_type": "refresh_token",
    })
    response = requests.post("https://www.googleapis.com/oauth2/v4/token",
                             data=data)
    if not response.ok:
      raise FrtbError(
          f"Authentication error: {response.content.decode('utf-8')}")

    return json.loads(response.content)["id_token"]

### Authentication

Instructions:

Send your Google enabled email address (GMail, Google Account or GSuite) to Avera-AI to be white listed for access to the FRTB-SA Service.

Once your email address has been white listed you can Authenticate this CoLab by:

1. Follow this link to the [Google Account Authentication page](https://accounts.google.com/o/oauth2/v2/auth?client_id=307640191694-hbvr0srnb1kdh5i713e0au7u0cnbdnai.apps.googleusercontent.com&response_type=code&scope=openid%20email&access_type=offline&redirect_uri=urn:ietf:wg:oauth:2.0:oob).
2. Sign-in using your Google enabled email. This step will be skipped ff the browser is already signed in with your Google account.
3. Copy code and paste the Authorization code into AUTHORIZATION_CODE variable in the cell below.

So the cell will look like:

      AUTHORIZATION_CODE = "your_authorization_code"

**Don't share this AUTHORIZATION_CODE with others.**

<i>Note: This Authorization process will change slightly in the next iteration of the Python client library so that there will be no need to copy and paste the AUTHORIZATION_CODE.</i>

In [ ]:
#@title Authetication
# Copy and Paste the AUTHORIZATION_CODE into the line below and then run this
# Cell.
# 
# Note:   Each authorization code can be used only once. If you run this cell
#         twice, you will have to use the link above again and use a different
#         authorization code.
AUTHORIZATION_CODE = "4/1gF3_iRSH0MyyU6Keuc7wV_0b9bqPqNB5EeqhB0pTPV3m4PwQKN_qCQ"
#    2.4. Initialize the Python FrtbClient:
CLIENT_ID="307640191694-hbvr0srnb1kdh5i713e0au7u0cnbdnai.apps.googleusercontent.com"
CLIENT_SECRET="8poN2ytn_C0RkZuvkNXL40Wl"

client = FrtbClient(CLIENT_ID, CLIENT_SECRET, AUTHORIZATION_CODE,
                    token_type=TokenType.AUTHORIZATION_CODE)

In [ ]:
#@title Client with refresh token (only run if need to refresh client authetication with updated refresh token produced after calling the aggregator first time below)
REFRESH_TOKEN = '1//05-3rcGkMpYrYCgYIARAAGAUSNwF-L9Irv-X35Nx31BTKKvqLhq5G3kmEHy0xTc-YLtWhU0sQn7rOqdCGA-rmsVoFuEe8NLqMcus'
client = FrtbClient(CLIENT_ID, CLIENT_SECRET, REFRESH_TOKEN,
                    token_type=TokenType.REFRESH)

# 1. Snapshot & Scatchpad Management

In [ ]:
for id in range(500,1200):
  try:
    client.delete_snapshot(snapshot_id=id)
  except:
    continue


**1.1 List Snapshots**

In [ ]:
#@title 1.1.1 List all Snapshots & Check Correct Number of Snapshots
snapshots_list = pd.DataFrame.from_records(client.list_snapshots(page_size=1000)['snapshots'])
display(snapshots_list.tail(10))

,name,id,title,createTime,publishTime,snapshotTime
147,frtb/snapshots/1182,1182,tK4N6drO3T,2020-07-07T07:39:35.696637Z,2020-07-07T07:39:37.972763Z,1970-01-01T00:00:00Z
148,frtb/snapshots/1183,1183,nsHiAUjEls,2020-07-07T08:18:54.039205Z,2020-07-07T08:18:57.065964Z,1970-01-01T00:00:00Z
149,frtb/snapshots/1184,1184,nnGhq5Hn8t,2020-07-07T08:35:46.221932Z,2020-07-07T08:35:48.911218Z,1970-01-01T00:00:00Z
150,frtb/snapshots/1185,1185,6PcNaka3wQ,2020-07-07T08:58:45.672593Z,2020-07-07T08:58:48.194671Z,1970-01-01T00:00:00Z
151,frtb/snapshots/1186,1186,XK1tdeOgca,2020-07-07T09:15:40.071235Z,2020-07-07T09:15:42.445050Z,1970-01-01T00:00:00Z
152,frtb/snapshots/1187,1187,snapshot_10k_5,2020-07-07T09:22:40.604275Z,2020-07-07T09:22:57.380455Z,1970-01-01T00:00:00Z
153,frtb/snapshots/1188,1188,snapshot_10k_6,2020-07-07T09:35:43.620626Z,2020-07-07T09:36:08.111397Z,1970-01-01T00:00:00Z
154,frtb/snapshots/1189,1189,snapshot_10k_7,2020-07-07T09:38:20.995085Z,2020-07-07T09:38:37.849884Z,1970-01-01T00:00:00Z
155,frtb/snapshots/1190,1190,snapshot_10k_8,2020-07-07T09:41:18.309253Z,2020-07-07T09:41:34.826893Z,1970-01-01T00:00:00Z
156,frtb/snapshots/1191,1191,snapshot_10k_9,2020-07-07T10:23:41.622333Z,2020-07-07T10:23:58.264559Z,1970-01-01T00:00:00Z


**1.2 Snapshot/Scratchpad Creation, Deletion**

In [ ]:
#@title 1.2.1 Create a new scratchpad
scratch_pad_title = tester_name + '_UAT_new_scratchpad'
scratch_pad = client.create_scratchpad(name='frtb/scratchpad/', title=scratch_pad_title)
display(scratch_pad)
len(scratch_pad)

{'id': 1082, 'name': 'frtb/scratchpads/1082'}

2

In [ ]:
#@title 1.2.2 Duplicate scratchpad should not be allowed
# Expected Error Code 6 with Message:  A snapshot or scratchpad already exists 
# with the title UAT3"
scratch_pad = client.create_scratchpad(name='frtb/scratchpad/', title=scratch_pad_title)

FrtbError: ignored

In [ ]:
#@title 1.2.3 Delete the scratchpad
client.delete_scratchpad(scratchpad_id=scratch_pad['id'])

{}

In [ ]:
#@title Recreated deleted snapshot, which should possible after deletion. 
# Should now be able to create scratchpad with name 'UAT3' again, as it has 
# been deleted, with a new scratchpad_id assigned. 
scratch_pad = client.create_scratchpad(name='frtb/scratchpad/', title=scratch_pad_title)
display(scratch_pad)

{'id': 1085, 'name': 'frtb/scratchpads/1085'}

In [ ]:
#@title 1.2.4 Delete a non-existing scatchpad
# This error message should be improved. 
client.delete_scratchpad(scratchpad_id=9999)

FrtbError: ignored

In [ ]:
#@title 1.2.5 Load 1000 test trades from Google Drive. 
downloaded = drive.CreateFile({'id': test_file_id_1})
test_trades = downloaded.GetContentString()
# JSON lines format. need to split out. 
test_trades_batches = test_trades.splitlines()

In [ ]:
#@title 1.2.6 Add batches to scratchpad.
for batch in test_trades_batches:
  client.add_to_scratchpad(scratchpad_id=scratch_pad['id'], 
                           trades_json = batch)

In [ ]:
 #@title 1.2.7 Publish scratchpad.
 snapshot = client.publish_scratchpad(scratchpad_id=scratch_pad['id'])
 client.list_trades(snapshot_id=932, portfolio_id='4')

{'trade': [{'accountId': 3,
   'bond': {'bankHolidays': 'US',
    'businessDayConvention': 'NO_ADJUSTMENT',
    'couponPaymentDays': [{'days': 19082},
     {'days': 18717},
     {'days': 18352},
     {'days': 17987}],
    'couponRate': 0.04,
    'currency': 'EUR',
    'dayCountConvention': 'ACTUAL_ACTUAL',
    'effectiveDate': {'days': 17622},
    'expirationDate': {'days': 19082},
    'faceValue': 10000,
    'quantity': 3,
    'tradeDate': {'days': 17622}},
   'counterparty': {'id': 'ECB'},
   'expirationDate': {'days': 19082},
   'labels': [{}],
   'tradeId': '12'}]}

In [ ]:
#@title 1.2.8 List portfolios for snapshot.
# This does not work yet, see b/
client.list_portfolios(snapshot_id=scratch_pad['id'])

### Check the content of pubished snapshot.

The underlying data is stored in BigQuery which allows adhoc queries of the raw data. This is demonstrated below by finding the number of individual trades and
sensitivites stored in a Snapshot.

<i>Note: access to the data via BigQuery requires additional permissions which KPMG staff haven't been granted.  So please treat this as an example.</i>

In [ ]:
#@title Authenticate with BigQuery.
# Authenticate with BigQuery.
# This is a separate Authentication step to the one above for access to the
# FRTB-SA Aggregator as it allows more detailed access to the underlying
# sensitivites, RRAO and DRC exposures.

# Running this cell will prompt you to follow a link and allow Bigquery
# access to your Google account.

# Internal check: access to the data via BigQuery requires 
#. additional permissions which KPMG staff haven't been granted. So please treat this as an example.

from google.colab import auth
auth.authenticate_user()

In [ ]:
#@title Check data for published snapshot
project_id = 'nomisma-database-staging'
snapshot_id = scratch_pad['id']
expected_no_trades = 100
summary_query = f"""
  SELECT
    IF(account_id IS NULL, 'Total', CAST(account_id AS STRING)) account_id,
    COUNT(DISTINCT trade_id) number_of_trades,
    COUNT(*) number_of_sensitivites
  FROM `FRTB.Snapshot_Sensitivities_{snapshot_id}`
  GROUP BY ROLLUP(account_id)
  ORDER BY 2 DESC"""


snapshot_summary = pandas_gbq.read_gbq(summary_query, 
                                            project_id=project_id,
                                            dialect='standard')
# Expect total of 1000 trades
trades_correct = snapshot_summary.number_of_trades[0] == expected_no_trades
display(f"""Correct number of trades: {trades_correct}""")
display(snapshot_summary)

'Correct number of trades: True'

,account_id,number_of_trades,number_of_sensitivites
0,None,100,882
1,77,3,26
2,0,3,26
3,8,3,27
4,79,3,27
...,...,...,...
61,80,1,9
62,6,1,9
63,65,1,9
64,67,1,9


In [ ]:
#@title 1.2.9 Upload incorrectly formatted JSON
# Expect an error.
downloaded = drive.CreateFile({'id': test_file_id_2})
test_trades = downloaded.GetContentString()
client.add_to_scratchpad(scratchpad_id=scratch_pad['id'], 
                         trades_json = test_trades)

FrtbError: ignored

In [ ]:
#@title 1.2.10 Upload data with incorrect RiskClass ('RANDOM_CLASS')
# Expect an error.
downloaded = drive.CreateFile({'id': test_file_id_3})
test_trades = downloaded.GetContentString()
client.add_to_scratchpad(scratchpad_id=324, 
                         trades_json = test_trades)
# Expect an error.

FrtbError: ignored

In [ ]:
#@title 1.2.11 Publish snapshot from JOSNL file
downloaded = drive.CreateFile({'id':test_file_id_1}) 
downloaded.GetContentFile('scratchpad-0.json1')  
scratchpad_jsonl = client.publish_snapshot_from_jsonl_file(
    path = 'scratchpad-0.json1',
    name='frtb/scratchpad/', 
    title='uat_kmpg_jsonl_3')
display(scratchpad_jsonl)

Creating a scratchpad
Uploading batch 1
Uploading batch 2
Uploading batch 3
Uploading batch 4
Uploading batch 5
Uploading batch 6
Uploading batch 7
Uploading batch 8
Uploading batch 9
Uploading batch 10
Publishing


1223

In [ ]:
#@title Check latest snapshot exists and match publish time 
snapshots_list = pd.DataFrame.from_records(client.list_snapshots(
    page_size=1000)['snapshots'])
display(snapshots_list.tail(10))

,name,id,title,createTime,publishTime,snapshotTime
126,frtb/snapshots/290,290,test_sensitivities4,2020-06-05T17:05:32.413646Z,2020-06-05T17:08:05.234176Z,1970-01-01T00:00:00Z
127,frtb/snapshots/291,291,test_sensitivities5,2020-06-05T17:34:20.182490Z,2020-06-05T17:36:54.947087Z,1970-01-01T00:00:00Z
128,frtb/snapshots/292,292,test_sensitivities6,2020-06-05T17:55:17.830097Z,2020-06-05T17:57:49.054058Z,1970-01-01T00:00:00Z
129,frtb/snapshots/293,293,test_sensitivities_10000_trades,2020-06-08T12:33:27.406700Z,2020-06-08T12:34:39.226740Z,1970-01-01T00:00:00Z
130,frtb/snapshots/294,294,test_sensitivities_50000_no_commod,2020-06-10T09:32:47.078552Z,2020-06-10T09:35:20.423261Z,1970-01-01T00:00:00Z
131,frtb/snapshots/325,325,fullrandom_2.6,2020-06-11T14:27:12.676929Z,2020-06-11T15:37:01.836687Z,1970-01-01T00:00:00Z
132,frtb/snapshots/326,326,no_commod_2.6,2020-06-11T18:04:50.482869Z,2020-06-11T20:19:32.012777Z,1970-01-01T00:00:00Z
133,frtb/snapshots/492,492,KPMG_two_trades_test,2020-06-24T14:27:34.624345Z,2020-06-24T14:27:39.941216Z,1970-01-01T00:00:00Z
134,frtb/snapshots/493,493,partitioning_no_commod_2.6,2020-06-24T18:38:10.988502Z,2020-06-24T19:44:51.570381Z,1970-01-01T00:00:00Z
135,frtb/snapshots/494,494,clustering_no_commod_2.6,2020-06-25T17:57:00.381143Z,2020-06-25T19:03:54.904355Z,1970-01-01T00:00:00Z


In [ ]:
#@title Check content of snapshot as expected
project_id = 'nomisma-database-staging'
snapshot_id = scratchpad_jsonl
expected_no_trades = 1000
summary_query = f"""
  SELECT
    IF(account_id IS NULL, 'Total', CAST(account_id AS STRING)) account_id,
    COUNT(DISTINCT trade_id) number_of_trades,
    COUNT(*) number_of_sensitivites
  FROM `FRTB.Snapshot_Sensitivities_{snapshot_id}`
  GROUP BY ROLLUP(account_id)
  ORDER BY 2 DESC"""


snapshot_summary = df = pandas_gbq.read_gbq(summary_query, 
                                            project_id=project_id,
                                            dialect='standard')
# Expect total of 1000 trades
trades_correct = snapshot_summary.number_of_trades[0] == expected_no_trades
display(f"""Correct number of trades: {trades_correct}""")
display(snapshot_summary)

'Correct number of trades: False'

,account_id,number_of_trades,number_of_sensitivites
0,None,100,882
1,77,3,26
2,0,3,26
3,8,3,27
4,79,3,27
...,...,...,...
61,80,1,9
62,6,1,9
63,65,1,9
64,67,1,9


In [ ]:
#@title 1.2.12 Delete the last snapshot & verify deletion
snapshot_id = scratchpad_jsonl
client.delete_snapshot(snapshot_id=snapshot_id)
snapshots_list = pd.DataFrame.from_records(client.list_snapshots(
    page_size=1000)['snapshots'])

In [ ]:
snapshot_found = (snapshot_id in snapshots_list.id)
print(f'Snapshot found: {snapshot_found}')

Snapshot found: False


**1.3 Snapshot/Scratchpad Manipulation**

In [ ]:
#@title 1.3.1 Label a snapshot
snapshot_id = scratch_pad['id']
client.label_snapshot(snapshot_id=snapshot_id, 
                      label={'key':'MONEYBANK','value':'Portfolio1'})

{}

In [ ]:
# Check that label has been added.
snapshots_list = pd.DataFrame.from_records(client.list_snapshots(page_size=1000)['snapshots'])
labeled_snapshot = snapshots_list.loc[snapshots_list['id'] == snapshot_id]
labeled_snapshot['labels']

,name,id,title,createTime,publishTime,snapshotTime,labels
162,frtb/snapshots/1085,1085,UAT_scratchpad,2020-07-02T21:04:47.397939Z,2020-07-02T21:05:07.243169Z,1970-01-01T00:00:00Z,"[{'key': 'MONEYBANK', 'value': 'Portfolio1'}]"


In [ ]:
#@title 1.3.2 Update snapshot label 
client.relabel_snapshot(snapshot_id=snapshot_id, 
                      label={'key':'MONEYBANK','value':'Portfolio42'})

{}

In [ ]:
# Check that label has been updated.
snapshots_list = pd.DataFrame.from_dict(client.list_snapshots(page_size=1000)['snapshots'])
labeled_snapshot = snapshots_list.loc[snapshots_list['id'] == snapshot_id]
labeled_snapshot['labels'].values[0][0]['key']

'MONEYBANK'

In [ ]:
#@title 1.3.3 Try update snapshot label with non-existing label key
# Expect an error. Label key must match existing key.
client.relabel_snapshot(snapshot_id=snapshot_id, 
                        label={'key':'NO_MONEYBANK','value':'Portfolio123'})

FrtbError: ignored

In [ ]:
#@title 1.3.4 Delete snapshot labels
client.unlabel_snapshot(snapshot_id=snapshot_id, label_key = 'MONEYBANK')

{}

In [ ]:
client.label_snapshot(snapshot_id=123456789, 
                      label={'key':'MONEYBANK','value':'Portfolio1'})

FrtbError: ignored

In [ ]:
# Check that labels have been deleted.
snapshots_list = pd.DataFrame.from_records(client.list_snapshots(page_size=1000)['snapshots'])
labeled_snapshot = snapshots_list.loc[snapshots_list['id'] == snapshot_id]
display(labeled_snapshot)
'labels' in labeled_snapshot.columns

False

In [ ]:
#@title 1.3.5 Retrieve trades from a snapshot
snapshot_id = scratch_pad['id']
account_id = 75
trade_id = 1
my_trades = client.get_trades(snapshot_id=snapshot_id, account_id=account_id, 
                              trade_id=trade_id)
display(my_trades)

{'trade': {'accountId': 75,
  'bond': {'bankHolidays': 'US',
   'businessDayConvention': 'NO_ADJUSTMENT',
   'couponPaymentDays': [{'days': 19483},
    {'days': 19118},
    {'days': 18753},
    {'days': 18388}],
   'couponRate': 0.04,
   'currency': 'USD',
   'dayCountConvention': 'ACTUAL_ACTUAL',
   'effectiveDate': {'days': 18023},
   'expirationDate': {'days': 19483},
   'faceValue': 10000,
   'quantity': 10,
   'tradeDate': {'days': 18023}},
  'counterparty': {'id': 'US_treasury'},
  'expirationDate': {'days': 19483},
  'labels': [{}],
  'tradeId': '1'}}

In [ ]:
#@title 1.3.6 Try retrieve non-existing trades from a snapshot
snapshot_id = snapshot_id
account_id = 81
trade_id = 424242
my_trades = client.get_trades(snapshot_id=snapshot_id, account_id=account_id, 
                              trade_id=trade_id)
# Produces sensible error message

FrtbError: ignored

In [ ]:
#@title 1.3.6 Try retrieve badly specified trade id.
snapshot_id = snapshot_id 
account_id = 81
trade_id = 'A'
my_trades = client.get_trades(snapshot_id=snapshot_id, account_id=account_id, 
                              trade_id=trade_id)
# Produces sensible error message

FrtbError: ignored

In [ ]:
#@title 1.3.7 List portfolios in a snapshot
client.list_portfolios(snapshot_id=snapshot_id)
# CHECK This. No working for me. Could just be my calling of API.

# 2. Calculation and Aggregation of Capitial Charges

In [ ]:
#@title 2.1 Capital Calculation Broken out by Risk Component
# Sensitivities-based Approach, DRC, and RRAO.
capital_charges = client.calculate_frtb_sa(snapshot_id)
cap_components = pd.DataFrame.from_records([{'Quantity': k, 'Value': v} for k, v in capital_charges.items() if k != 'capitalSensitivityBreakdown'])
display(cap_components)

,Quantity,Value
0,reportingCurrency,GBP
1,capitalTotal,"14,216,598.12"
2,capitalSensitivity,"8,787,581.02"
3,capitalRrao,"3,506.59"
4,capitalDrc,"5,425,510.51"
5,parameterSet,frtb/parameter_sets/1


In [ ]:
#@title 2.2 Capital Calculation Broken out by Sensitivity Risk Class.
# The 7 risk classes
risk_classes = ['FX', 'INTEREST_RATE', 
                'CREDIT_NON_SEC', 
                'CREDIT_SEC_CTP', 
                'CREDIT_SEC_NON_CTP', 
                'COMMOD', 'EQUITIES']
capital_sensitivities = pd.DataFrame(
    capital_charges['capitalSensitivityBreakdown'])
cap_risk_class = capital_sensitivities[capital_sensitivities.riskClass.isin(
    risk_classes)]
cap_risk_class = capital_sensitivities[capital_sensitivities.aggregationType ==
    'SENSITIVITY_AGGREGATION_BY_CLASS']
display(cap_risk_class.drop(['riskMeasure', 'correlationScenario'], axis=1))

,aggregationType,capitalSensitivity,riskClass
6,SENSITIVITY_AGGREGATION_BY_CLASS,"2,058,319.97",COMMOD
10,SENSITIVITY_AGGREGATION_BY_CLASS,"982,086.98",CREDIT_SEC_CTP
14,SENSITIVITY_AGGREGATION_BY_CLASS,"1,160,995.16",CREDIT_NON_SEC
18,SENSITIVITY_AGGREGATION_BY_CLASS,"390,010.16",CREDIT_SEC_NON_CTP
22,SENSITIVITY_AGGREGATION_BY_CLASS,"1,263,105.87",EQUITIES
26,SENSITIVITY_AGGREGATION_BY_CLASS,"1,691,514.86",FX
30,SENSITIVITY_AGGREGATION_BY_CLASS,"1,241,548.03",INTEREST_RATE


In [ ]:
#@title 2.3 Capital Calculation Broken out by correlation Scenario and Risk Class
cap_risk_class = capital_sensitivities[capital_sensitivities.riskClass.isin(
    risk_classes)]
cap_risk_class = capital_sensitivities[capital_sensitivities.aggregationType ==
    'SENSITIVITY_AGGREGATION_BY_CORRELATION_CLASS']
cap_risk_corr = cap_risk_class.drop(['aggregationType', 'riskMeasure'], axis=1)
pivot = cap_risk_corr.pivot_table(index='riskClass', columns=['correlationScenario'], 
                          values='capitalSensitivity', aggfunc = 'sum',
                          margins=True,)
display(pivot)
pivot.columns.values[:-1]

correlation_scenarios =['HIGH_CORRELATIONS', 'LOW_CORRELATIONS', 'MEDIUM_CORRELATIONS']
pivot[pivot.index.get_level_values('riskClass') == "All"][correlation_scenarios]

array(['HIGH_CORRELATIONS', 'LOW_CORRELATIONS', 'MEDIUM_CORRELATIONS'],
      dtype=object)

In [ ]:
capital_charges = client.calculate_frtb_sa(
        snapshot_id=1223)
capital_sensitivities = pd.DataFrame(
        capital_charges['capitalSensitivityBreakdown'])
risk_classes = ['FX', 'INTEREST_RATE', 'CREDIT_NON_SEC', 'CREDIT_SEC_CTP',
                    'CREDIT_SEC_NON_CTP', 'COMMOD', 'EQUITIES']
cap_risk_class = capital_sensitivities[
        capital_sensitivities.riskClass.isin(risk_classes)]
aggregation_type = 'SENSITIVITY_AGGREGATION_BY_CORRELATION_CLASS'
cap_risk_class = capital_sensitivities[
capital_sensitivities.aggregationType == aggregation_type]
cap_risk_corr = cap_risk_class.drop(['aggregationType', 'riskMeasure'], axis=1)
pivot = cap_risk_corr.pivot_table(index='riskClass',
                                  columns=['correlationScenario'],
                                  values='capitalSensitivity',
                                  aggfunc='sum',
                                  margins=True)
scenarios = ['HIGH_CORRELATIONS', 'LOW_CORRELATIONS', 'MEDIUM_CORRELATIONS']
corr_totals = pivot[(pivot.index.get_level_values('riskClass') == 'All')]
max(corr_totals[scenarios].iloc[0])
pivot.columns.values

array(['HIGH_CORRELATIONS', 'LOW_CORRELATIONS', 'MEDIUM_CORRELATIONS',
       'All'], dtype=object)

In [ ]:
#@title 2.4 Trade Overrides
# Compute origial Capital as before
capital_original = client.calculate_frtb_sa(snapshot_id)['capitalTotal']

# Display the results.
results = [{
    'Scenario' : 'Original data from snapshot',
    'snapshot_id'  : snapshot_id,
    'capital' : capital_original
  }]
display(f"Total Capital: {capital_original}")

'Total Capital: 14216598.123847296'

In [ ]:
 #@title 2.4.1 Override: Addtrade with only RROA charge (trade_id=123) in portfolio 0. 
trade_id = 7
portfolio_id = 0
overrides = [{
    'trade' : {'tradeId': trade_id},

    'rraoRiskFactors' : [{'riskType' : 'EXOTIC_UNDERLYING','notional' : 10000000}]
}]
capital_new = client.calculate_frtb_sa_with_overrides(
    snapshot_id=snapshot_id, portfolio_id=portfolio_id, 
    overrides=overrides)['capitalTotal']
results.append({
    'Scenario' : f'Overriding trade_id = {trade_id}',
    'snapshot_id'  : snapshot_id,
    'capital' : capital_new
  })
display(pd.DataFrame.from_records(results))
# Expect to increase capital by 10,000,000 x 0.1 = 1,000,000, as no currently 
# associated RROA for this this trade in Portfolio 0.

,Scenario,snapshot_id,capital
0,Original data from snapshot,474,"14,216,598.12"
1,Overriding trade_id = 7,474,"14,171,491.45"


In [ ]:
 #@title 2.4.2 Add trade with only DRC charge (trade_id=456) in portfolio 0. 
trade_id = 777
overrides = [{
    'trade' : {'tradeId': trade_id},
    'drcRiskFactors':[{"drcRiskFactorId":{"portfolioType":"SECURITIZATION_NON_CTP",
                                          "defaultRiskDetails":{
                                              "drcSecNonCtp":{"underlyingPortfolioId":"3",
                                                              "attachmentPoint":0.02,
                                                              "detachmentPoint":0.07,
                                                              "maturity":8.852640797373716,
                                                              "bucket":"CORPORATE",
                                                              "ratioOfRiskWeightedCapitalToTotalExposure":0.976553020407678,
                                                              "delinquencyKnownRatio":1.0,
                                                              "isSenior":True}}},
                    "notional":10000,
                    "pnl":-600}]
}]

capital_new_drc = client.calculate_frtb_sa_with_overrides(
    snapshot_id=snapshot_id, portfolio_id=portfolio_id, 
    overrides=overrides)['capitalTotal']
results.append({
    'Scenario' : f'Overriding trade_id = {trade_id}',
    'snapshot_id'  : snapshot_id,
    'capital' : capital_new_drc
  })
display(pd.DataFrame.from_records(results))

,Scenario,snapshot_id,capital
0,Original data from snapshot,474,"14,216,598.12"
1,Overriding trade_id = 7,474,"14,171,491.45"
2,Overriding trade_id = 777,474,"14,334,098.12"


In [ ]:
#@title Create sensitivity override trade
trade_id = 789
override_trade_sensitivity = [{
    "trade":{"tradeId":trade_id,
             "counterparty":{"id":"US_treasury"},
             "accountId":79,"expirationDate":{"days":19461},
             "bond":{"tradeDate":{"days":18366},
                     "effectiveDate":{"days":18366},
                    "couponPaymentDays":[{"days":19461},{"days":19096},{"days":18731}],
                    "dayCountConvention":"ACTUAL_ACTUAL",
                    "currency":"USD",
                    "quantity":85.0,
                    "couponRate":0.04,
                   "faceValue":10000.0,
                   "bankHolidays":"US",
                   "expirationDate":{"days":19461},
                   "businessDayConvention":"NO_ADJUSTMENT"},
                   "sensitivityRiskFactors":[{"id":{"riskClass":"CREDIT_NON_SEC","riskMeasure":"VEGA",
                                                    "riskFactor":{"vegaCsrNonSec":{
                                                        "bucket":"HY_NR_GOVT_NON_FINANCIALS",
                                                        "riskFactor":{"deltaRiskFactor":{
                                                            "issuerCurve":{"issuerId":94,
                                                                           "curveType":"CDS"},
                                                                           "tenor":"ONE_YEAR"}, 
                                                                      "optionMaturity":"THREE_YEAR"}}}},
                                              "value":100000.0}]}
                               }]

In [ ]:
 #@title 2.4.3 Add trade incurring sensitivity-based charge (trade_id=789) in portfolio 0. 
trade_id = 789
portfolio_id = 0
capital_new_sensitivity = client.calculate_frtb_sa_with_overrides(
    snapshot_id=snapshot_id, 
    portfolio_id=portfolio_id, 
    overrides=override_trade_sensitivity)['capitalTotal']
results.append({
    'Scenario' : f'Overriding trade_id = {trade_id}',
    'snapshot_id'  : snapshot_id,
    'capital' : capital_new_sensitivity
  })
display(pd.DataFrame.from_records(results))
# Why is total Capital not updated after overrode for sensitivity trade?

,Scenario,snapshot_id,capital
0,Original data from snapshot,474,"14,216,598.12"
1,Overriding trade_id = 7,474,"14,171,491.45"
2,Overriding trade_id = 777,474,"14,334,098.12"
3,Overriding trade_id = 789,474,"14,216,598.12"


In [ ]:
#@title 2.4.4 Try add override trade with mis-specified risk factor
trade_id = 123
overrides = [{
    'trade' : {'tradeId': trade_id},

    'AbcRiskFactors' : [{'riskType' : 'EXOTIC_UNDERLYING', 'notional' : 10000000}]
}]
capital_new = client.calculate_frtb_sa_with_overrides(
    snapshot_id=snapshot_id, portfolio_id=portfolio_id, 
    overrides=overrides)['capitalTotal']
results.append({
    'Scenario' : f'Overriding trade_id = {trade_id}',
    'snapshot_id'  : snapshot_id,
    'capital' : capital_new
  })
display(pd.DataFrame.from_records(results))
# I would expect error checking identifying 'AbcRiskFactors' as misspecified.
# Similarly if specify 'RraoRiskFactors' correctly and 'totalmoney' instead of
# 'notional'. 

,Scenario,snapshot_id,capital
0,Original data from snapshot,474,"14,216,598.12"
1,Overriding trade_id = 7,474,"14,171,491.45"
2,Overriding trade_id = 777,474,"14,334,098.12"
3,Overriding trade_id = 789,474,"14,216,598.12"
4,Overriding trade_id = 123,474,"14,216,598.12"


# 3. Test Calculations
We test basic calculations using small test portfolios.

In [ ]:
#@title Authenticate with BigQuery.
# Authenticate with BigQuery.
# This is a separate Authentication step to the one above for access to the
# FRTB-SA Aggregator as it allows more detailed access to the underlying
# sensitivites, RRAO and DRC exposures.

# Running this cell will prompt you to follow a link and allow Bigquery
# access to your Google account.

# Internal check: access to the data via BigQuery requires 
#. additional permissions which KPMG staff haven't been granted. So please treat this as an example.

from google.colab import auth
auth.authenticate_user()

In [ ]:
#@title Upload portfolio with two trades. 
scratchpad_title_2trades = tester_name + '_two_trades_test_2'
scratch_pad = client.create_scratchpad(name='frtb/scratchpad/', title=scratchpad_title_2trades)
downloaded = drive.CreateFile({'id': two_trade_portfolio})
test_trades = downloaded.GetContentString()
display(scratch_pad)

{'id': 1208, 'name': 'frtb/scratchpads/1208'}

In [ ]:
#@title 3.1. Upload Test Portfolio
# Upload portfolio with two trades. 
client.add_to_scratchpad(scratchpad_id=scratch_pad['id'], 
                         trades_json = test_trades)
# Publish Snapshot of test trades
test_snapshot = client.publish_scratchpad(scratchpad_id=scratch_pad['id'])

# Display Data
from google.colab import auth
auth.authenticate_user()
project_id = 'nomisma-database-staging'
snapshot_id = scratch_pad['id']
summary_query = f"""
  SELECT
   *
  FROM `FRTB.Snapshot_Sensitivities_{snapshot_id}`
  """
snapshot_summary = pandas_gbq.read_gbq(summary_query, 
                                       project_id=project_id,
                                       dialect='standard')

In [ ]:
#@title 3.2 Display Test Portfolio
printmd("**Snapshot Data:**")
print('\n\n\n\n\n\n\n\n\n')
snapshot_summary_flat = flat_table.normalize(snapshot_summary)
column_names = snapshot_summary_flat.columns
new_column_names = [x.replace('sensitivity_struct.risk_factor.', '').replace('sensitivity_struct.', '') for x in column_names]
snapshot_summary_flat.columns = new_column_names
snapshot_summary_flat = snapshot_summary_flat.drop(['index', 'security', 'frtb_id'], axis=1)
display(snapshot_summary_flat.style.set_table_styles(
    [dict(selector="th",props=[('max-width', '80px')]),
        dict(selector="th.col_heading",
                 props=[("writing-mode", "vertical-rl"), 
                        ('transform', 'rotateZ(-10deg)'),
                        (("font-size", "75%"))
                        ])]
))

# Display Trades
printmd("**Snapshot Trade Details:**")
print('\n')
account_id = 1
trade_ids = [1, 2]
trade_details = pd.DataFrame(None)
for trade in trade_ids:
  display(client.get_trades(snapshot_id=scratch_pad['id'],
                            account_id=account_id, 
                            trade_id=trade))

**Snapshot Data:**

,trade_id,counterparty,security_type_id,account_id,expiration_days_since_epoch,value,frtb_id_hash,curvature_fx.curvature_scenario,curvature_fx.bucket,curvature_commod.risk_factor.delta_risk_factor.delivery_id,curvature_commod.risk_factor.delta_risk_factor.maturity,curvature_commod.risk_factor.delta_risk_factor.symbol_id,curvature_commod.curvature_scenario,curvature_commod.bucket,curvature_equities.risk_factor.delta_risk_factor.market_type,curvature_equities.risk_factor.delta_risk_factor.symbol_id,curvature_equities.curvature_scenario,curvature_equities.bucket,curvature_csr_sec_ctp.risk_factor.delta_risk_factor.issuer_curve.curve_type,curvature_csr_sec_ctp.risk_factor.delta_risk_factor.issuer_curve.issuer_id,curvature_csr_sec_ctp.risk_factor.delta_risk_factor.tenor,curvature_csr_sec_ctp.curvature_scenario,curvature_csr_sec_ctp.bucket,curvature_csr_sec_non_ctp.risk_factor.delta_risk_factor.curve_type,curvature_csr_sec_non_ctp.risk_factor.delta_risk_factor.tenor,curvature_csr_sec_non_ctp.risk_factor.delta_risk_factor.tranche_id,curvature_csr_sec_non_ctp.curvature_scenario,curvature_csr_sec_non_ctp.bucket,curvature_csr_non_sec.risk_factor.delta_risk_factor.issuer_curve.curve_type,curvature_csr_non_sec.risk_factor.delta_risk_factor.issuer_curve.issuer_id,curvature_csr_non_sec.risk_factor.delta_risk_factor.tenor,curvature_csr_non_sec.curvature_scenario,curvature_csr_non_sec.bucket,curvature_girr.curvature_scenario,curvature_girr.bucket,vega_fx.option_maturity,vega_fx.bucket,vega_commod.risk_factor.delta_risk_factor.delivery_id,vega_commod.risk_factor.delta_risk_factor.maturity,vega_commod.risk_factor.delta_risk_factor.symbol_id,vega_commod.risk_factor.option_maturity,vega_commod.bucket,vega_equities.risk_factor.delta_risk_factor.market_type,vega_equities.risk_factor.delta_risk_factor.symbol_id,vega_equities.risk_factor.option_maturity,vega_equities.bucket,vega_csr_sec_ctp.risk_factor.delta_risk_factor.issuer_curve.curve_type,vega_csr_sec_ctp.risk_factor.delta_risk_factor.issuer_curve.issuer_id,vega_csr_sec_ctp.risk_factor.delta_risk_factor.tenor,vega_csr_sec_ctp.risk_factor.option_maturity,vega_csr_sec_ctp.bucket,vega_csr_sec_non_ctp.risk_factor.delta_risk_factor.curve_type,vega_csr_sec_non_ctp.risk_factor.delta_risk_factor.tenor,vega_csr_sec_non_ctp.risk_factor.delta_risk_factor.tranche_id,vega_csr_sec_non_ctp.risk_factor.option_maturity,vega_csr_sec_non_ctp.bucket,vega_csr_non_sec.risk_factor.delta_risk_factor.issuer_curve.curve_type,vega_csr_non_sec.risk_factor.delta_risk_factor.issuer_curve.issuer_id,vega_csr_non_sec.risk_factor.delta_risk_factor.tenor,vega_csr_non_sec.risk_factor.option_maturity,vega_csr_non_sec.bucket,vega_girr.risk_factor.residual_maturity,vega_girr.risk_factor.option_maturity,vega_girr.bucket,delta_fx.bucket,delta_commod.risk_factor.delivery_id,delta_commod.risk_factor.maturity,delta_commod.risk_factor.symbol_id,delta_commod.bucket,delta_equities.risk_factor.market_type,delta_equities.risk_factor.symbol_id,delta_equities.bucket,delta_csr_sec_ctp.risk_factor.issuer_curve.curve_type,delta_csr_sec_ctp.risk_factor.issuer_curve.issuer_id,delta_csr_sec_ctp.risk_factor.tenor,delta_csr_sec_ctp.bucket,delta_csr_sec_non_ctp.risk_factor.curve_type,delta_csr_sec_non_ctp.risk_factor.tenor,delta_csr_sec_non_ctp.risk_factor.tranche_id,delta_csr_sec_non_ctp.bucket,delta_csr_non_sec.risk_factor.issuer_curve.curve_type,delta_csr_non_sec.risk_factor.issuer_curve.issuer_id,delta_csr_non_sec.risk_factor.tenor,delta_csr_non_sec.bucket,delta_girr.risk_factor.curve_risk_factor.tenor,delta_girr.risk_factor.curve_risk_factor.curve_type,delta_girr.risk_factor.risk_factor_case,delta_girr.risk_factor.special_risk_factor,delta_girr.bucket,risk_factor_case,risk_measure,risk_class,trade_labels.value,trade_labels.key
0,1,US_treasury,5,1,19410,50.000000,3500125147364547581,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,None,None,None,None,None,nan,nan,nan,nan,nan,None,None,None,None,None,nan,nan,nan,nan,None,None,None,None,None,None,None,None,N

**Snapshot Trade Details:**

{'trade': {'accountId': 1,
  'bond': {'bankHolidays': 'US',
   'businessDayConvention': 'NO_ADJUSTMENT',
   'couponPaymentDays': [{'days': 19410},
    {'days': 19230},
    {'days': 19050},
    {'days': 18870},
    {'days': 18690},
    {'days': 18510}],
   'couponRate': 0.04,
   'currency': 'USD',
   'dayCountConvention': 'ACTUAL_ACTUAL',
   'effectiveDate': {'days': 18315},
   'expirationDate': {'days': 19410},
   'faceValue': 100,
   'quantity': 5,
   'tradeDate': {'days': 18315}},
  'counterparty': {'id': 'US_treasury'},
  'expirationDate': {'days': 19410},
  'labels': [{}],
  'tradeId': '1'}}

{'trade': {'accountId': 1,
  'bond': {'bankHolidays': 'US',
   'businessDayConvention': 'NO_ADJUSTMENT',
   'couponPaymentDays': [{'days': 19476},
    {'days': 19111},
    {'days': 18746},
    {'days': 18381}],
   'couponRate': 0.04,
   'currency': 'GBP',
   'dayCountConvention': 'ACTUAL_ACTUAL',
   'effectiveDate': {'days': 18016},
   'expirationDate': {'days': 19476},
   'faceValue': 100,
   'quantity': 10,
   'tradeDate': {'days': 18016}},
  'counterparty': {'id': 'UK_treasury'},
  'expirationDate': {'days': 19476},
  'labels': [{}],
  'tradeId': '2'}}

In [ ]:
#@title Export Portfolio Data to Google Drive to Create Golden Set Data.
f = drive.CreateFile({'title': 'sensitivies.csv'})
f.SetContentString(snapshot_summary_flat.to_csv(header=True,index=False))
f.Upload()
print('Uploaded file with ID {}'.format(f.get('id')))

Uploaded file with ID 1MmSas0MzbGh1k5EHijGPJaNP103Dhp9c


In [ ]:
#@title 3.3 Sensitivities Calculation Tests
# Only sensitivity based charges expected for this portfolio
snapshot_id = scratch_pad['id']
capital_charges = client.calculate_frtb_sa(snapshot_id)
cap_components = pd.DataFrame.from_records([{'Quantity': k, 'Value': v} for k, v in capital_charges.items() if k != 'capitalSensitivityBreakdown'])
display(cap_components)

In [ ]:
risk_classes = ['FX', 'INTEREST_RATE', 'CREDIT_NON_SEC', 'CREDIT_SEC_CTP', 'CREDIT_SEC_NON_CTP', 'COMMOD', 'EQUITIES']
capital_sensitivities = pd.DataFrame(capital_charges['capitalSensitivityBreakdown'])
cap_risk_class = capital_sensitivities[capital_sensitivities.riskClass.isin(risk_classes)]
aggregation_type = 'SENSITIVITY_AGGREGATION_BY_CLASS'
cap_risk_class = capital_sensitivities[capital_sensitivities.aggregationType == aggregation_type]
sum(cap_risk_class.capitalSensitivity)

317.7001858675242

In [ ]:
#@title 3.3.1 Sensitivities Calculation Test: sum over riskClass equal to capitalTotal
risk_classes = ['DELTA', 'VEGA', 'CURVATURE']
cap_breakdown = pd.DataFrame.from_dict([{'Quantity': k, 'Value': v} for k, v in capital_charges.items() if k == 'capitalSensitivityBreakdown'])
# cap_breakdown = flat_table.normalize(cap_breakdown)
cap_breakdown = pd.json_normalize(json.loads(json.dumps(cap_breakdown.Value[0])))
cap_breakdown = cap_breakdown[cap_breakdown['correlationScenario'] == 'HIGH_CORRELATIONS']
cap_breakdown = cap_breakdown[cap_breakdown['riskMeasure'].isin(risk_classes)]
# Aggregate by riksMeasure
display(cap_breakdown)
# Sum over risk measures:
total = round(cap_breakdown['capitalSensitivity'].sum(), 12)
total_test = total == round(cap_components.Value[1],12)
printmd(f"**Sum over risk measures is: {total}**")
printmd(f"**Same as capitalTotal: {total_test}**")

from pandas.io import json as json_pd



,aggregationType,correlationScenario,capitalSensitivity,riskClass,riskMeasure
33,SENSITIVITY_AGGREGATION_BY_CORRELATION_MEASURE,HIGH_CORRELATIONS,211.21,NaN,VEGA
37,SENSITIVITY_AGGREGATION_BY_CORRELATION_MEASURE,HIGH_CORRELATIONS,100.87,NaN,CURVATURE
41,SENSITIVITY_AGGREGATION_BY_CORRELATION_MEASURE,HIGH_CORRELATIONS,5.62,NaN,DELTA


**Sum over risk measures is: 317.700185867524**

**Same as capitalTotal: True**

3.3.2 Verify Sensitivities based calculation
The above trade sensitivities per risk factor were exported and the capital charge computed independently using source document at: https://www.bis.org/bcbs/publ/d457.htm


For FX no optional discretionary adjustment was made, as per default in the AVERA aggregator.

The calculation is outlined in this document: 
[go/avera-frtb-sensitivity-golden-data](https://docs.google.com/spreadsheets/d/1v_r6PQ-rihb6Rv5PNTxl_ZJ0q7QNDskGUffK8nplFw0/edit#gid=2085681443). 

This gives 317.70, matching the above calculation.  

In [ ]:
# Upload portfolio with three trades from drive
scratchpad_title_3trades = tester_name + '_three_trades_test_1'
scratch_pad = client.create_scratchpad(name='frtb/scratchpad/', title=scratchpad_title_3trades)
downloaded = drive.CreateFile({'id': three_trade_portfolio})
test_trades2 = downloaded.GetContentString()
display(scratch_pad)

{'id': 1195, 'name': 'frtb/scratchpads/1195'}

In [ ]:
#@title 3.4.1 DRC Exposure Calculation Tests (Publish New Test Data)
# Publish Snapshot of test trades
client.add_to_scratchpad(scratchpad_id=scratch_pad['id'], 
                         trades_json = test_trades2)
# Publish Snapshot of test trades
test_snapshot = client.publish_scratchpad(scratchpad_id=scratch_pad['id'])
display(test_snapshot)

{'name': 'operations/publications/1195'}

In [ ]:
#@title 3.4.2 Get Snapshot Summary
from google.colab import auth
auth.authenticate_user()
project_id = 'nomisma-database-staging'
snapshot_id = scratch_pad['id']
summary_query = f"""
  SELECT
   *
  FROM `FRTB.Snapshot_Sensitivities_{snapshot_id}`
  """
snapshot_summary = pandas_gbq.read_gbq(summary_query, 
                                       project_id=project_id,
                                       dialect='standard')

In [ ]:
#@title 3.4.2 Display Test Portfolio
printmd("**Snapshot Data:**")
print('\n\n\n\n\n\n\n\n\n')
snapshot_summary_flat = flat_table.normalize(snapshot_summary)
column_names = snapshot_summary_flat.columns
new_column_names = [x.replace('sensitivity_struct.risk_factor.', '').replace('sensitivity_struct.', '') for x in column_names]
snapshot_summary_flat.columns = new_column_names
snapshot_summary_flat = snapshot_summary_flat.drop(['index', 'security', 'frtb_id'], axis=1)
display(snapshot_summary_flat.style.set_table_styles(
    [dict(selector="th",props=[('max-width', '80px')]),
        dict(selector="th.col_heading",
                 props=[("writing-mode", "vertical-rl"), 
                        ('transform', 'rotateZ(-10deg)'),
                        (("font-size", "75%"))
                        ])]
))

# Display Trades
printmd("**Snapshot Trade Details:**")
print('\n')
account_id = 1
trade_ids = [1, 2, 3]
trade_details = pd.DataFrame(None)
for trade in trade_ids:
  display(client.get_trades(snapshot_id=scratch_pad['id'],
                            account_id=account_id, 
                            trade_id=trade))

**Snapshot Data:**

,trade_id,counterparty,security_type_id,account_id,expiration_days_since_epoch,value,frtb_id_hash,curvature_fx.curvature_scenario,curvature_fx.bucket,curvature_commod.risk_factor.delta_risk_factor.delivery_id,curvature_commod.risk_factor.delta_risk_factor.maturity,curvature_commod.risk_factor.delta_risk_factor.symbol_id,curvature_commod.curvature_scenario,curvature_commod.bucket,curvature_equities.risk_factor.delta_risk_factor.market_type,curvature_equities.risk_factor.delta_risk_factor.symbol_id,curvature_equities.curvature_scenario,curvature_equities.bucket,curvature_csr_sec_ctp.risk_factor.delta_risk_factor.issuer_curve.curve_type,curvature_csr_sec_ctp.risk_factor.delta_risk_factor.issuer_curve.issuer_id,curvature_csr_sec_ctp.risk_factor.delta_risk_factor.tenor,curvature_csr_sec_ctp.curvature_scenario,curvature_csr_sec_ctp.bucket,curvature_csr_sec_non_ctp.risk_factor.delta_risk_factor.curve_type,curvature_csr_sec_non_ctp.risk_factor.delta_risk_factor.tenor,curvature_csr_sec_non_ctp.risk_factor.delta_risk_factor.tranche_id,curvature_csr_sec_non_ctp.curvature_scenario,curvature_csr_sec_non_ctp.bucket,curvature_csr_non_sec.risk_factor.delta_risk_factor.issuer_curve.curve_type,curvature_csr_non_sec.risk_factor.delta_risk_factor.issuer_curve.issuer_id,curvature_csr_non_sec.risk_factor.delta_risk_factor.tenor,curvature_csr_non_sec.curvature_scenario,curvature_csr_non_sec.bucket,curvature_girr.curvature_scenario,curvature_girr.bucket,vega_fx.option_maturity,vega_fx.bucket,vega_commod.risk_factor.delta_risk_factor.delivery_id,vega_commod.risk_factor.delta_risk_factor.maturity,vega_commod.risk_factor.delta_risk_factor.symbol_id,vega_commod.risk_factor.option_maturity,vega_commod.bucket,vega_equities.risk_factor.delta_risk_factor.market_type,vega_equities.risk_factor.delta_risk_factor.symbol_id,vega_equities.risk_factor.option_maturity,vega_equities.bucket,vega_csr_sec_ctp.risk_factor.delta_risk_factor.issuer_curve.curve_type,vega_csr_sec_ctp.risk_factor.delta_risk_factor.issuer_curve.issuer_id,vega_csr_sec_ctp.risk_factor.delta_risk_factor.tenor,vega_csr_sec_ctp.risk_factor.option_maturity,vega_csr_sec_ctp.bucket,vega_csr_sec_non_ctp.risk_factor.delta_risk_factor.curve_type,vega_csr_sec_non_ctp.risk_factor.delta_risk_factor.tenor,vega_csr_sec_non_ctp.risk_factor.delta_risk_factor.tranche_id,vega_csr_sec_non_ctp.risk_factor.option_maturity,vega_csr_sec_non_ctp.bucket,vega_csr_non_sec.risk_factor.delta_risk_factor.issuer_curve.curve_type,vega_csr_non_sec.risk_factor.delta_risk_factor.issuer_curve.issuer_id,vega_csr_non_sec.risk_factor.delta_risk_factor.tenor,vega_csr_non_sec.risk_factor.option_maturity,vega_csr_non_sec.bucket,vega_girr.risk_factor.residual_maturity,vega_girr.risk_factor.option_maturity,vega_girr.bucket,delta_fx.bucket,delta_commod.risk_factor.delivery_id,delta_commod.risk_factor.maturity,delta_commod.risk_factor.symbol_id,delta_commod.bucket,delta_equities.risk_factor.market_type,delta_equities.risk_factor.symbol_id,delta_equities.bucket,delta_csr_sec_ctp.risk_factor.issuer_curve.curve_type,delta_csr_sec_ctp.risk_factor.issuer_curve.issuer_id,delta_csr_sec_ctp.risk_factor.tenor,delta_csr_sec_ctp.bucket,delta_csr_sec_non_ctp.risk_factor.curve_type,delta_csr_sec_non_ctp.risk_factor.tenor,delta_csr_sec_non_ctp.risk_factor.tranche_id,delta_csr_sec_non_ctp.bucket,delta_csr_non_sec.risk_factor.issuer_curve.curve_type,delta_csr_non_sec.risk_factor.issuer_curve.issuer_id,delta_csr_non_sec.risk_factor.tenor,delta_csr_non_sec.bucket,delta_girr.risk_factor.curve_risk_factor.tenor,delta_girr.risk_factor.curve_risk_factor.curve_type,delta_girr.risk_factor.risk_factor_case,delta_girr.risk_factor.special_risk_factor,delta_girr.bucket,risk_factor_case,risk_measure,risk_class,trade_labels.value,trade_labels.key
0,1,US_treasury,5,1,19461,67966.439127,8785558697471345290,nan,nan,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,nan,nan,None,None,nan,nan,nan,nan,nan,No

**Snapshot Trade Details:**

{'trade': {'accountId': 1,
  'bond': {'bankHolidays': 'US',
   'businessDayConvention': 'NO_ADJUSTMENT',
   'couponPaymentDays': [{'days': 19461}, {'days': 19096}, {'days': 18731}],
   'couponRate': 0.04,
   'currency': 'USD',
   'dayCountConvention': 'ACTUAL_ACTUAL',
   'effectiveDate': {'days': 18366},
   'expirationDate': {'days': 19461},
   'faceValue': 10000,
   'quantity': 85,
   'tradeDate': {'days': 18366}},
  'counterparty': {'id': 'US_treasury'},
  'expirationDate': {'days': 19461},
  'labels': [{}],
  'tradeId': '1'}}

{'trade': {'accountId': 1,
  'bond': {'bankHolidays': 'US',
   'businessDayConvention': 'NO_ADJUSTMENT',
   'couponPaymentDays': [{'days': 19847},
    {'days': 19667},
    {'days': 19487},
    {'days': 19307},
    {'days': 19127},
    {'days': 18947},
    {'days': 18767},
    {'days': 18587},
    {'days': 18407},
    {'days': 18227}],
   'couponRate': 0.04,
   'currency': 'USD',
   'dayCountConvention': 'ACTUAL_ACTUAL',
   'effectiveDate': {'days': 18022},
   'expirationDate': {'days': 19847},
   'faceValue': 10000,
   'quantity': 68,
   'tradeDate': {'days': 18022}},
  'counterparty': {'id': 'US_treasury'},
  'expirationDate': {'days': 19847},
  'labels': [{}],
  'tradeId': '2'}}

{'trade': {'accountId': 1,
  'bond': {'bankHolidays': 'US',
   'businessDayConvention': 'NO_ADJUSTMENT',
   'couponPaymentDays': [{'days': 18440},
    {'days': 18075},
    {'days': 17710},
    {'days': 17345}],
   'couponRate': 0.04,
   'currency': 'GBP',
   'dayCountConvention': 'ACTUAL_ACTUAL',
   'effectiveDate': {'days': 16980},
   'expirationDate': {'days': 18440},
   'faceValue': 10000,
   'quantity': 6,
   'tradeDate': {'days': 16980}},
  'counterparty': {'id': 'UK_treasury'},
  'expirationDate': {'days': 18440},
  'labels': [{}],
  'tradeId': '3'}}

In [ ]:
#@title Export Portfolio Data to Google Drive to Create Golden Set Data.
f = drive.CreateFile({'title': 'sensitivies2.csv'})
f.SetContentString(snapshot_summary_flat.to_csv(header=True,index=False))
f.Upload()
print('Uploaded file with ID {}'.format(f.get('id')))

Uploaded file with ID 1x85ZKHZibeNB5VYUanr3dndLSUydJYfN


In [ ]:
#@title 3.4.3 Capital Calculation Broken out by Risk Component
# Sensitivities-based Approach, DRC, and RRAO.
snapshot_id = scratch_pad['id']
capital_charges = client.calculate_frtb_sa(snapshot_id=snapshot_id)
cap_components = pd.DataFrame.from_records([{'Quantity': k, 'Value': v} for k, v in capital_charges.items() if k != 'capitalSensitivityBreakdown'])
display(cap_components)

,Quantity,Value
0,reportingCurrency,GBP
1,capitalTotal,"1,184,757.28"
2,capitalSensitivity,"411,105.67"
3,capitalRrao,115.84
4,capitalDrc,"773,535.77"
5,parameterSet,frtb/parameter_sets/1


In [ ]:
#@title 3.4.4 Sensitivities Calculation Test: sum over riskClass equal to capitalTotal
from pandas.io import json as json_pd
risk_classes = ['DELTA', 'VEGA', 'CURVATURE']
cap_components = pd.DataFrame.from_records([{'Quantity': k, 'Value': v} for k, v in capital_charges.items() if k == 'capitalSensitivityBreakdown'])
flat_table = json_pd.json_normalize(
        json.loads(json.dumps(cap_components.Value[0])))
cap_breakdown = flat_table[flat_table['aggregationType'] == 'SENSITIVITY_AGGREGATION_BY_MEASURE']
cap_breakdown = cap_breakdown[cap_breakdown['riskMeasure'].isin(risk_classes)]
cap_breakdown = cap_breakdown[cap_breakdown['aggregationType'] == 'SENSITIVITY_AGGREGATION_BY_MEASURE']
cap_breakdown = cap_breakdown[cap_breakdown['riskMeasure'].isin(risk_classes)]
# Aggregate by riskMeasure
display(cap_breakdown)
# This shows discrepancies to manual calculations for the portfolio file 
# imported for above snapshot: 
# https://drive.google.com/file/d/1u5Yx1k1trtFBinDfamMmkOWPRiTncB9Z/view?usp=sharing

# Manual calculations: Wiesner
# Verifier: TBC
#https://goto.google.com/avera-frtb-sensitivity-golden-data
cap_breakdown = flat_table[flat_table['aggregationType'] == 'SENSITIVITY_AGGREGATION_BY_MEASURE']
cap_breakdown = cap_breakdown[cap_breakdown['riskMeasure'].isin(risk_classes)]

flat_table[flat_table['aggregationType'] == 'SENSITIVITY_AGGREGATION_TOTAL']['capitalSensitivity']


43   317.70
Name: capitalSensitivity, dtype: float64

In [ ]:
#@title 4.1 Test Repeatability of Results

# We upload a portfolio of 10000 trades covering sensitivity-based method, DRC and RRAO.
# It covers all risk measures and risk classes, and most risk factors.
scratchpad_title_repeat = tester_name + '_test_repeatability'
downloaded = drive.CreateFile({'id': test_file_id_4})
downloaded.GetContentFile('scratchpad-4.json1')  
# Add test trades to scratchpad
snapshot_repeat = client.publish_snapshot_from_jsonl_file(
    path = 'scratchpad-4.json1',
    name='frtb/scratchpad/', 
    title=scratchpad_title_repeat)

display(snapshot_repeat)
display(scratch_pad)

In [ ]:
# TEMP: Unittests testing
scratchpad_title_repeat = 'xyz123'
downloaded = drive.CreateFile({'id': test_file_id_5})
downloaded.GetContentFile('scratchpad-test.json1')  
# Add test trades to scratchpad
snapshot_repeat = client.publish_snapshot_from_jsonl_file(
    path = 'scratchpad-test.json1',
    name='frtb/scratchpad/', 
    title=scratchpad_title_repeat)

capital_original = client.calculate_frtb_sa(1180)['capitalTotal']
capital_original


trade_id = 201
portfolio_id = 0
overrides = [{'trade': {'tradeId': trade_id},
              'rraoRiskFactors': [
                                  {'riskType': 'EXOTIC_UNDERLYING', 
                                   'notional': 10000000}]
              }]
capital_new = client.calculate_frtb_sa_with_overrides(
    snapshot_id=1180, portfolio_id=portfolio_id,
    overrides=overrides)['capitalTotal']